**Importa las bibliotecas necesarias.**

In [1]:
import psycopg2
from psycopg2 import Error # Importar la clase Error
from psycopg2 import sql
import pandas as pd

**Función para ejecutar una consulta SQL**

In [2]:
# Función para ejecutar una consulta SQL
def execute_query(conn, query, params=None):
    """
    Ejecuta una consulta SQL con manejo de errores y soporte para parámetros.
    """
    try:
        with conn.cursor() as cursor:
            cursor.execute(query, params)  # Ejecuta la consulta con o sin parámetros
            conn.commit()  # Confirma los cambios si no hay errores
            print("Query executed successfully")
            if cursor.description:
                # Si la consulta devuelve resultados, obtén los datos
                return cursor.fetchall()
    except psycopg2.Error as err:
        conn.rollback()  # Restaura la transacción si ocurre un error
        print(f"Error executing query: {err}")
        return None  # Devuelve None en caso de error


**Conexión a la base de datos de PostgreSQL data_powergen**

In [3]:
# Conexión a la base de datos
def create_database_connection():
    conn = None
    try:
        conn = psycopg2.connect(
            database="data_powergen", 
            user="datafundamental_owner", 
            password="uozci0wQGpA6", 
            host="ep-solitary-pond-a2p3evpo.eu-central-1.aws.neon.tech", 
            sslmode="require", 
            port=5432
        )
        print("Connected to 'data_powergen' successfully.")
    except psycopg2.Error as err:
        print(f"Error: {err}")
    return conn

**Crea la tabla resultados_join y la lista de consultas a ejecutar.**

In [4]:
# Lista de consultas a ejecutar
sql_queries = {
    "create_resultados_join": """
        CREATE TABLE IF NOT EXISTS resultados_join AS
        SELECT 
            h.id_equipo,
            COUNT(h.id_orden) AS total_ordenes_mantenimiento,
            AVG(r.temperatura_c) AS promedio_temperatura,
            AVG(r.vibracion_mm_s) AS promedio_vibracion,
            AVG(r.presion_bar) AS promedio_presion
        FROM historicos_ordenes h
        LEFT JOIN registros_condiciones r
        ON h.id_equipo = r.id_equipo
        GROUP BY h.id_equipo;
    """,
    "total_ordenes_por_equipo": """
        SELECT id_equipo, COUNT(id_orden) AS total_ordenes
        FROM historicos_ordenes
        GROUP BY id_equipo;
    """,
    "duracion_promedio_vida_util": """
        SELECT AVG(vida_util_anios) AS duracion_promedio_vida_util
        FROM caracteristicas_equipos;
    """,
    "costo_promedio_correctivo": """
        SELECT AVG(costo_mantenimiento) AS costo_promedio_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo';
    """,
    "equipo_mas_fallas": """
        SELECT id_equipo, COUNT(id_orden) AS total_correctivo
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo'
        GROUP BY id_equipo
        ORDER BY total_correctivo DESC
        LIMIT 1;
    """,
    "temperatura_resumen": """
        SELECT id_equipo, AVG(temperatura_c) AS promedio_temp, MAX(temperatura_c) AS max_temp
        FROM registros_condiciones
        GROUP BY id_equipo;
    """,
    "ordenes_correctivas_equipo": """
        SELECT id_orden, id_equipo, costo_mantenimiento, tiempo_reparacion_horas
        FROM historicos_ordenes
        WHERE tipo_mantenimiento = 'Correctivo' AND id_equipo = %s;
    """,
    "dias_vibracion_temperatura": """
        SELECT *
        FROM registros_condiciones
        WHERE vibracion_mm_s > 3.0 AND temperatura_c > 60;
    """,
    "tiempo_promedio_mantenimiento": """
        SELECT id_equipo, AVG(EXTRACT(EPOCH FROM AGE(fecha, LAG(fecha) OVER (PARTITION BY id_equipo ORDER BY fecha)))/3600) AS tiempo_promedio_horas
        FROM historicos_ordenes
        GROUP BY id_equipo;
    """,
    "suma_horas_acumuladas": """
        SELECT id_equipo, SUM(horas_acumuladas) AS total_horas_acumuladas
        FROM registros_condiciones
        GROUP BY id_equipo;
    """
}


sql_queries


{'create_resultados_join': '\n        CREATE TABLE IF NOT EXISTS resultados_join AS\n        SELECT \n            h.id_equipo,\n            COUNT(h.id_orden) AS total_ordenes_mantenimiento,\n            AVG(r.temperatura_c) AS promedio_temperatura,\n            AVG(r.vibracion_mm_s) AS promedio_vibracion,\n            AVG(r.presion_bar) AS promedio_presion\n        FROM historicos_ordenes h\n        LEFT JOIN registros_condiciones r\n        ON h.id_equipo = r.id_equipo\n        GROUP BY h.id_equipo;\n    ',
 'total_ordenes_por_equipo': '\n        SELECT id_equipo, COUNT(id_orden) AS total_ordenes\n        FROM historicos_ordenes\n        GROUP BY id_equipo;\n    ',
 'duracion_promedio_vida_util': '\n        SELECT AVG(vida_util_anios) AS duracion_promedio_vida_util\n        FROM caracteristicas_equipos;\n    ',
 'costo_promedio_correctivo': "\n        SELECT AVG(costo_mantenimiento) AS costo_promedio_correctivo\n        FROM historicos_ordenes\n        WHERE tipo_mantenimiento = 'Corr

**Ejecuta el código (conectar y consultar)**

In [5]:
# Ejecución del script
if __name__ == "__main__":
    # Conectar a la base de datos
    db_conn = create_database_connection()

    # Ejecutar cada consulta
    for query_name, query in sql_queries.items():
        print(f"Executing query: {query_name}")
        result = execute_query(db_conn, query)
        if result:  # Si la consulta devuelve resultados, imprimirlos
            print(result)

    # Cerrar la conexión
    db_conn.close()


Connected to 'data_powergen' successfully.
Executing query: create_resultados_join
Query executed successfully
Executing query: total_ordenes_por_equipo
Query executed successfully
[('E005', 502), ('E013', 496), ('E011', 483), ('E019', 516), ('E001', 487), ('E003', 506), ('E020', 516), ('E015', 495), ('E016', 478), ('E006', 473), ('E010', 537), ('E008', 531), ('E014', 464), ('E007', 485), ('E004', 539), ('E017', 532), ('E018', 498), ('E009', 507), ('E012', 470), ('E002', 485)]
Executing query: duracion_promedio_vida_util
Query executed successfully
[(Decimal('19.0000000000000000'),)]
Executing query: costo_promedio_correctivo
Query executed successfully
[(Decimal('2711.4088353413654618'),)]
Executing query: equipo_mas_fallas
Query executed successfully
[('E010', 272)]
Executing query: temperatura_resumen
Query executed successfully
[('E005', Decimal('50.4801952277657267'), Decimal('80.0')), ('E013', Decimal('51.2583956043956044'), Decimal('80.0')), ('E011', Decimal('49.4766952789699571